<a href="https://colab.research.google.com/github/veillette/jupyterNotebooks/blob/main/Advanced%20Lab/Error%20Propagation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Error propagation
### Function that performs error propagation for any function of multiple variables

In this general method:

- You define the error_propagation function, which takes the function to be evaluated (func), a list of variables (variables), and a list of their associated uncertainties (uncertainties).

- Inside the function, it calculates the function value (func_value) and then computes the squared partial derivatives with respect to each variable.

- The total uncertainty is calculated as the square root of the sum of the squared partial derivatives.

- The function returns both the function value and the total uncertainty.

- You can use this error_propagation function with any function that takes multiple variables and their uncertainties to calculate the propagated uncertainty for the function's output.

In [6]:
import numpy as np

def error_propagation(func, variables, uncertainties):
    # Calculate the function value
    func_value = func(*variables)

    # Calculate the squared partial derivatives
    squared_partial_derivatives_times_square_uncertainties = []
    for i in range(len(variables)):
        perturbed_vars = variables.copy()
        perturbed_vars[i] += uncertainties[i]
        partial_derivative = (func(*perturbed_vars) - func_value) / uncertainties[i]
        squared_partial_derivatives_times_square_uncertainties.append(partial_derivative**2*uncertainties[i]**2)

    # Calculate the total uncertainty
    total_uncertainty = np.sqrt(sum(squared_partial_derivatives_times_square_uncertainties))

    return func_value, total_uncertainty

The code above uses a function value $$f(x_1,x_2,x_3, \cdots)$$.
For each variable, it calculates the squared partial derivative:

  -It creates a copy of the variables and perturbs one variable by its uncertainty.
  -It calculates the partial derivative using the finite difference method:
    $$ \frac{\partial f}{\partial x_i} \approx \frac{f(x_1, \ldots, x_i + \delta x_i, \ldots, x_n) - f(x_1, \ldots, x_n)}{\delta x_i} $$
  -It squares the partial derivative and stores it in a list.


 Finally, it calculates the total uncertainty using the formula:
$$ \sigma_f = \sqrt{\sum_{i=1}^n \left(\frac{\partial f}{\partial x_i}\right)^2 \sigma_{x_i}^2} $$
where $\sigma_f$ is the total uncertainty, $\frac{\partial f}{\partial x_i}$ are the partial derivatives, and $\sigma_{x_i}$ are the individual uncertainties.

The function returns both the function value and the total uncertainty.

In [ ]:
# Example usage:
# Define a function that takes two variables
def example_function(x, y):
    return x**2 * y

# Define the values and uncertainties (in absolute values) for the variables
variables = [2.0, 3.0]
uncertainties = [0.1, 0.2]

# Perform error propagation
result, error = error_propagation(example_function, variables, uncertainties)

# Print the result and total uncertainty
print(f"Result = {result}")
print(f"Total Uncertainty = {error}")

In [8]:
# Example usage:
# Define a function that takes two variables
def example_function(b):
    return np.log(2)/b

# Define the values and uncertainties (in absolute values) for the variables
variables = [0.0045]
uncertainties = [0.00003]

# Perform error propagation
result, error = error_propagation(example_function, variables, uncertainties)

# Print the result and total uncertainty
print(f"Result = {result}")
print(f"Total Uncertainty = {error}")

Result = 154.03270679109897
Total Uncertainty = 1.020084150934423


In [ ]:
np.log